In [23]:
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

TRBC:
1) Same industry
2) Highly related
3) Mightly related
4) No relation


total=100%
majority=50%-100% excl
minority= less than 50%

Advanced preclean
- share_at_announcment: yes or no dummy OK
- shares_acquired: the above plus acquired shares to find category: full, majority or minority
- announcement data: year, seasonality OK
- target_nation: need to add new col from downloaded dataset (‘continent’)
- target_status: public or not (dummy)
- attitude: dummy
- consideration_offered: cash or not

In [3]:
!pwd

/Users/manonlaffly/code/cobergmann/MA_PREDICTOR/notebooks


In [125]:
data= pd.read_csv('/Users/manonlaffly/code/cobergmann/MA_PREDICTOR/MA_PREDICTOR/data/ma_detailed_data_car.csv', parse_dates=['announcement_date'])

In [126]:
data

,id,announcement_date,deal_value,acquiror_total_assets,acquiror_name,acquiror_ticker,target_name,target_nation,acquiror_nation,target_status,...,shares_acquired,consideration_offered,attitude,acquisition_technique,acquiror_financial_advisor,target_financial_advisor,purpose,acquiror_code,target_code,car
0,1631732020,2005-01-01,12.00,3160.41,Plains All American Pipeline LP,PAA,Shell Pipeline Co LP-Crude Oil Pipeline Assets...,United States,United States,Subsidiary,...,100.00,Cash|Cash Only,Friendly,Financial Acquiror|Divestiture,NaN,NaN,Strengthen existing operations/expand presence...,50102030,50103030,0.006854
1,1629274020,2005-04-01,153.67,6166.05,SuperValu Inc,SVU,Total Logistics Inc,United States,United States,Public,...,100.00,Cash Only|Cash,Friendly,Tender Offer|Tender Merger,Lazard,William Blair & Co,Strengthen existing operations/expand presence...,54301020,52405030,0.000000
2,1631278020,2005-11-01,89.16,11127.95,Fulton Financial Corp,FULT,"SVB Financial Services Inc, Sommerville,New Je...",United States,United States,Public,...,100.00,Cash|Cash|Common Stock|Common Stock,Friendly,Stock Swap,NaN,Danielson Associates Inc,Expand presence in new/foreign markets,55101010,55101010,0.000000
3,1631266040,2005-12-01,427.54,1116.90,Cleveland-Cliffs Inc,CLF,Portman Ltd,Australia,United States,Public,...,80.39,Cash|Cash,Friendly,Tender Offer,Wilson HTM|Fleet National Bank|Fifth Third Bank,Gresham Partners|Azure Capital,Strengthen existing operations/expand presence...,51201020,51201020,0.000000
4,1632176020,2005-01-14,16.00,1722.27,Thomas & Betts Corp,TNB,Southern Monopole & Utilities Co,United States,United States,Subsidiary,...,100.00,Cash|Cash Only,Friendly,Divestiture,NaN,NaN,Acquire competitors technology/strategic assets,52102030,51201020,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3325,3774828020,2021-07-19,625.00,5969.17,Bill.com Holdings Inc,BILL,Invoice2go Inc,United States,United States,Private,...,100.00,Cash|Common Stock,Friendly,Stock Swap,NaN,NaN,Strengthen operations,57201020,57201020,0.088395
3326,3778220020,2021-07-19,335.00,1240.31,Rapid7 Inc,RPD,Intsights Cyber Intelligence Ltd,United States,United States,Private,...,100.00,Common Stock|Cash,Friendly,Financial Acquiror,Foros,NaN,Strengthen operations,57201020,57201020,0.068755
3327,3777682020,2021-07-28,1450.00,6234.00,Avery Dennison Corp,AVY,Vestcom International Inc,United States,United States,Subsidiary,...,100.00,Cash|Cash Only,Friendly,Divestiture,Goldman Sachs & Co,RBC Capital Markets LLC|Jefferies LLC|Robert W...,"Create synergies, eliminate duplicate services...",51302020,53302010,0.022186
3328,3790420020,2021-05-08,280.00,1229.78,Insmed Inc,INSM,Motus Biosciences Inc,United States,United States,Private,...,100.00,Earnout|Common Stock|Cash,Friendly,Stock Swap,NaN,NaN,Strengthen operations,56202010,56202010,0.000000


In [127]:
data.columns

Index(['id', 'announcement_date', 'deal_value', 'acquiror_total_assets',
       'acquiror_name', 'acquiror_ticker', 'target_name', 'target_nation',
       'acquiror_nation', 'target_status', 'acquiror_industry',
       'target_industry', 'shares_at_announcement', 'shares_acquired',
       'consideration_offered', 'attitude', 'acquisition_technique',
       'acquiror_financial_advisor', 'target_financial_advisor', 'purpose',
       'acquiror_code', 'target_code', 'car'],
      dtype='object')

In [128]:
data['shares_at_announcement'] = data['shares_at_announcement'].astype(str).astype(float)

In [129]:
data.dtypes

id                                     int64
announcement_date             datetime64[ns]
deal_value                           float64
acquiror_total_assets                float64
acquiror_name                         object
acquiror_ticker                       object
target_name                           object
target_nation                         object
acquiror_nation                       object
target_status                         object
acquiror_industry                     object
target_industry                       object
shares_at_announcement               float64
shares_acquired                      float64
consideration_offered                 object
attitude                              object
acquisition_technique                 object
acquiror_financial_advisor            object
target_financial_advisor              object
purpose                               object
acquiror_code                          int64
target_code                            int64
car       

In [130]:
data['shares_at_announcement'] = data['shares_at_announcement'].apply(lambda x: "No" if x==0. else "Yes")

In [139]:
data['nat_vs_cross']=(data['target_nation'] == data['acquiror_nation']).map({True: 'national', False: 'crossborder'})

In [142]:
data['year'] = data['announcement_date'].dt.strftime('%Y')

In [144]:
data['month'] = data['announcement_date'].dt.strftime('%m')

In [145]:
data

,id,announcement_date,deal_value,acquiror_total_assets,acquiror_name,acquiror_ticker,target_name,target_nation,acquiror_nation,target_status,...,acquisition_technique,acquiror_financial_advisor,target_financial_advisor,purpose,acquiror_code,target_code,car,nat_vs_cross,year,month
0,1631732020,2005-01-01,12.00,3160.41,Plains All American Pipeline LP,PAA,Shell Pipeline Co LP-Crude Oil Pipeline Assets...,United States,United States,Subsidiary,...,Financial Acquiror|Divestiture,NaN,NaN,Strengthen existing operations/expand presence...,50102030,50103030,0.006854,national,2005,01
1,1629274020,2005-04-01,153.67,6166.05,SuperValu Inc,SVU,Total Logistics Inc,United States,United States,Public,...,Tender Offer|Tender Merger,Lazard,William Blair & Co,Strengthen existing operations/expand presence...,54301020,52405030,0.000000,national,2005,04
2,1631278020,2005-11-01,89.16,11127.95,Fulton Financial Corp,FULT,"SVB Financial Services Inc, Sommerville,New Je...",United States,United States,Public,...,Stock Swap,NaN,Danielson Associates Inc,Expand presence in new/foreign markets,55101010,55101010,0.000000,national,2005,11
3,1631266040,2005-12-01,427.54,1116.90,Cleveland-Cliffs Inc,CLF,Portman Ltd,Australia,United States,Public,...,Tender Offer,Wilson HTM|Fleet National Bank|Fifth Third Bank,Gresham Partners|Azure Capital,Strengthen existing operations/expand presence...,51201020,51201020,0.000000,crossborder,2005,12
4,1632176020,2005-01-14,16.00,1722.27,Thomas & Betts Corp,TNB,Southern Monopole & Utilities Co,United States,United States,Subsidiary,...,Divestiture,NaN,NaN,Acquire competitors technology/strategic assets,52102030,51201020,0.000000,national,2005,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3325,3774828020,2021-07-19,625.00,5969.17,Bill.com Holdings Inc,BILL,Invoice2go Inc,United States,United States,Private,...,Stock Swap,NaN,NaN,Strengthen operations,57201020,57201020,0.088395,national,2021,07
3326,3778220020,2021-07-19,335.00,1240.31,Rapid7 Inc,RPD,Intsights Cyber Intelligence Ltd,United States,United States,Private,...,Financial Acquiror,Foros,NaN,Strengthen operations,57201020,57201020,0.068755,national,2021,07
3327,3777682020,2021-07-28,1450.00,6234.00,Avery Dennison Corp,AVY,Vestcom International Inc,United States,United States,Subsidiary,...,Divestiture,Goldman Sachs & Co,RBC Capital Markets LLC|Jefferies LLC|Robert W...,"Create synergies, eliminate duplicate services...",51302020,53302010,0.022186,national,2021,07
3328,3790420020,2021-05-08,280.00,1229.78,Insmed Inc,INSM,Motus Biosciences Inc,United States,United States,Private,...,Stock Swap,NaN,NaN,Strengthen operations,56202010,56202010,0.000000,national,2021,05


In [148]:
data['shares_at_announcement'] = data['shares_at_announcement'].apply(lambda x: "Full" if x==100.0 else "Majority" if  (100 > x >=50) else "Minority" if x<50)


SyntaxError: invalid syntax (1021117429.py, line 1)

In [68]:
def extract_year(df):
    #import ipdb; ipdb.set_trace()
    year = df['announcement_date'].dt.strftime('%Y')
    month = df['announcement_date'].dt.strftime('%m')
    return pd.DataFrame({'year': year, 'month': month})

In [69]:
annoncement_date = FunctionTransformer(extract_year)

In [70]:
preprocessor = ColumnTransformer([
    ('extract_year', annoncement_date, ['announcement_date'])])

In [71]:
preprocessor.fit_transform(data)

array([['2005', '01'],
       ['2005', '04'],
       ['2005', '11'],
       ...,
       ['2021', '07'],
       ['2021', '05'],
       ['2021', '08']], dtype=object)

In [ ]:
# Preprocessor
preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, ['age', 'bmi']),
    ('cat_transformer', OneHotEncoder(), ['region', 'smoker']),
    ('children_scaler', CustomScaler(shrink_factor=3), ['children'])])